# Generators

- Writing a class-based iterator requires `__iter__()` and `__next__()`, plus manual state management and `StopIteration` handling.  
- Generator functions let you express the same logic in plain Python functions, using `yield` to produce values **one at a time**.  
- Any function with `yield` becomes a generator: calling it returns a **generator object (an iterator)** without running its body immediately.  

In [1]:
def count_up_to(limit):
    """Generates numbers from 1 up to (and including) the limit.

    Args:
        limit (int): The upper limit for counting.

    Returns:
        generator(int): The generator to lazily count up to limit.
    """
    print("Generator function started...")
    n = 1
    while n <= limit:
        print(f"Yielding {n}")
        yield n
        print(f"Resumed after yielding {n}.")
        n += 1
    print("Generator function finished.")

# Generator function immediately returns Generator object (Iterator) that we can iterate manually
count_gen = count_up_to(3)
print(f"Returned object: {count_gen} of type {type(count_gen)}")

# The value is only yield on calling next()
print("First call to next outside of for loop.")
num = next(count_gen)
print(num)

# Iterate the remaining values (for loop calls next() internally)
print("Remaining output from for loop.")
for number in count_gen:
    print(number)
    

Returned object: <generator object count_up_to at 0x104365380> of type <class 'generator'>
First call to next outside of for loop.
Generator function started...
Yielding 1
1
Remaining output from for loop.
Resumed after yielding 1.
Yielding 2
2
Resumed after yielding 2.
Yielding 3
3
Resumed after yielding 3.
Generator function finished.


## Generator Functions & the `yield` Keyword

- A function becomes a generator by including `yield`; no other boilerplate is needed.  
- Calling a generator function returns an object that implements `__iter__()` and `__next__()`.  
- The code inside runs only when iteration begins (e.g., in a `for` loop or via `next()`). The `for loop` **calls** `next()` **internally**.

In [8]:
def filter_evens(data):
    """Yield only the even items from the input sequence.

    Args:
        data (iterable(int or float)): The data to iterate through and filter.

    Returns:
        generator(int or float): A generator object that yields the even items.
    """
    print("filter_evens: starting")
    for item in data:
        if item % 2 == 0:
            print(f"filter_evens: yielding {item}")
            # yield returns the value and goes to suspended state,
            # then wakes up and continues from where it left in the next iteration.
            # `yield` which returns one item at a time which makes our computation more memory efficient
            yield item
    print("filter_evens: finished")

evens_from_range = filter_evens(range(6))
print(f"Generator object created: {evens_from_range}")
for num in evens_from_range:
    print(f"Received even: {num}")

# List Comprehension
def using_list_comprehension(data):
    # The list is build eagerly in memory before it is returned and the entire list is returned
    # unlike generator which only returns one value at a time (lazy evaluation)
    return [item for item in data if item % 2 == 0]

evens = using_list_comprehension((0,1,2,3,4,5))
print(f"Using list comprehension: {evens}")



Generator object created: <generator object filter_evens at 0x104366340>
filter_evens: starting
filter_evens: yielding 0
Received even: 0
filter_evens: yielding 2
Received even: 2
filter_evens: yielding 4
Received even: 4
filter_evens: finished
Using list comprehension: [0, 2, 4]


## How `yield` Works: Pause and Resume

- On each `next()` or **loop iteration**, execution runs until it hits `yield`, returns the value, then pauses with all **local state intact**.  
- The next `next()` call resumes immediately after the `yield`, preserving variables and the instruction pointer.  
- When the function ends (no more `yield`), a `StopIteration` is raised automatically.  

In [9]:
def demo_three_yields():
    """Demonstrate how having multiple yield statements work."""
    print("Generator started")
    yield 1
    print("Generator resumed after yielding 1.")
    yield 2
    print("Generator resumed after yielding 2.")
    yield 3
    print("Generator finished.")

demo_gen = demo_three_yields()

print(next(demo_gen))
print(next(demo_gen))
print(next(demo_gen))
# print(next(demo_gen)) # raises a StopIteration Exception because there are no more yields

Generator started
1
Generator resumed after yielding 1.
2
Generator resumed after yielding 2.
3


## Generator State

- Generators keep their local variables alive between yields, making explicit state objects unnecessary.  
- This persistent state allows infinite or long-running sequences without full data storage.  

In [10]:
count_gen = count_up_to(5)

print("First call to next outside of for loop.")
print(next(count_gen))

print("Second call to next outside of for loop - now the value yielded is 2.")
print(next(count_gen))

print("Remaining output from for loop - prints from 3 onwards.")
for number in count_gen:
    print(number)

First call to next outside of for loop.
Generator function started...
Yielding 1
1
Second call to next outside of for loop - now the value yielded is 2.
Resumed after yielding 1.
Yielding 2
2
Remaining output from for loop - prints from 3 onwards.
Resumed after yielding 2.
Yielding 3
3
Resumed after yielding 3.
Yielding 4
4
Resumed after yielding 4.
Yielding 5
5
Resumed after yielding 5.
Generator function finished.


In [4]:
def count_up_to(limit):
    """Generates numbers from 1 up to (and including) the limit.

    Args:
        limit (int): The upper limit for counting.

    Returns:
        generator(int): The generator to lazily count up to limit.
    """
    print("Generator function started...")
    n = 1
    while n <= limit:
        print(f"Yielding {n}")
        yield n
        print(f"Resumed after yielding {n}.")
        n += 1
    print("Generator function finished.")

count_gen = count_up_to(3)

# Since generators have state, using the same generator object in nested loops can lead to issues.
# The inner for loop will complete the iteration, and the outer for loop will have a single pass
# since both the loops are using the same generator.
for num in count_gen:
    for num2 in count_gen:
        print(f" - {num}:{num2}")

print()
print("---------- Using separate generators for outer and inner loops ----------")

# The solution to this is to use separate generator objects.
for num in count_up_to(3):
    for num2 in count_up_to(3):
        print(f" - {num}:{num2}")

Generator function started...
Yielding 1
Resumed after yielding 1.
Yielding 2
 - 1:2
Resumed after yielding 2.
Yielding 3
 - 1:3
Resumed after yielding 3.
Generator function finished.

---------- Using separate generators for outer and inner loops ----------
Generator function started...
Yielding 1
Generator function started...
Yielding 1
 - 1:1
Resumed after yielding 1.
Yielding 2
 - 1:2
Resumed after yielding 2.
Yielding 3
 - 1:3
Resumed after yielding 3.
Generator function finished.
Resumed after yielding 1.
Yielding 2
Generator function started...
Yielding 1
 - 2:1
Resumed after yielding 1.
Yielding 2
 - 2:2
Resumed after yielding 2.
Yielding 3
 - 2:3
Resumed after yielding 3.
Generator function finished.
Resumed after yielding 2.
Yielding 3
Generator function started...
Yielding 1
 - 3:1
Resumed after yielding 1.
Yielding 2
 - 3:2
Resumed after yielding 2.
Yielding 3
 - 3:3
Resumed after yielding 3.
Generator function finished.
Resumed after yielding 3.
Generator function finished

## Exhaustion

- Once a generator’s code path completes (falls off the end or hits `return`), further `next()` calls immediately raise `StopIteration`.  
- A `for loop` over an exhausted generator does nothing on subsequent passes—you must call the function again for a fresh iterator.  

In [5]:
def count_up_to(limit):
    """Generates numbers from 1 up to (and including) the limit.

    Args:
        limit (int): The upper limit for counting.

    Returns:
        generator(int): The generator to lazily count up to limit.
    """
    print("Generator function started...")
    n = 1
    while n <= limit:
        print(f"Yielding {n}")
        yield n
        print(f"Resumed after yielding {n}.")
        n += 1
    print("Generator function finished.")


count_gen = count_up_to(2)
print(next(count_gen))
print(next(count_gen))

try:
    print(next(count_gen)) # will raise StopIteration exception
except StopIteration:
    print("Generator function raised StopIteration exception")

# Nothing will happen because the generator is already exhausted
for number in count_gen:
    print(number)

Generator function started...
Yielding 1
1
Resumed after yielding 1.
Yielding 2
2
Resumed after yielding 2.
Generator function finished.
Generator function raised StopIteration exception
